In [ ]:
!pip install google-generativeai
!pip install google-cloud-secret-manager

In [ ]:
from IPython import get_ipython
from IPython.display import Markdown, display

In [ ]:
from google.cloud import secretmanager

def access_secret_version(resource_name):
    client = secretmanager.SecretManagerServiceClient()
    response = client.access_secret_version(request={"name": resource_name})
    return response.payload.data.decode("UTF-8")

# Your secret's resource name
resource_name = "projects/843008103000/secrets/gemini_api/versions/1"

# Access the secret
apiKey = access_secret_version(resource_name)



In [ ]:
import google.generativeai as genai
genai.configure(api_key=apiKey)
model = genai.GenerativeModel("gemini-1.5-flash")
# response = model.generate_content("Explain how AI works")
# print(response.text)
welcome_prompt = "You are a excellent developer in life science and healthcare research. The mission is to advice researcher with limited coding experience. Please format your response in markdown by default."

In [ ]:
def explain(cell_number):
    """Return the content of the specified cell number."""
    ipython = get_ipython()  # Get the current IPython instance
    
    # Ensure the cell number is valid
    if cell_number < len(ipython.user_ns['In']):
        content = "Based on the input. Please concisely comment this code to explain each line. Ignore the run cell magic and just focus on the code or error " + ipython.user_ns['In'][cell_number] + "Do not add additional code"
        response = model.generate_content(welcome_prompt + content)

        return display(Markdown(response.text))
        # Return the content of the cell
        #return ipython.user_ns['In'][cell_number]
    else:
        # Error message for invalid cell number
        return "Cell number out of range."
    

    
def modify(cell_number,modification):
    """Return the content of the specified cell number."""
    ipython = get_ipython()  # Get the current IPython instance
    
    # Ensure the cell number is valid
    if cell_number < len(ipython.user_ns['In']):
        prompt = "Please modify the code "  + ipython.user_ns['In'][cell_number] + " to accomplish " + modification + "Ignore the run cell magic and just focus on the code. Assume all library has been loaded. Return only code."
        response = model.generate_content(prompt)

        return create_new_cell("%%R\n\n" + response.text)

        # Return the content of the cell
        #return ipython.user_ns['In'][cell_number]
    else:
        # Error message for invalid cell number
        return "Cell number out of range."

def propose(suggest):
    response = model.generate_content("Please suggest code to accomplish " + suggest + ". Return only code.")
    #print(response.choices[0].message.content)
    #return display(Markdown(response.choices[0].message.content))
    return create_new_cell("\n\n" + response.text)
    
def create_new_cell(contents):
    shell = get_ipython()
    shell.set_next_input(contents, replace=False)


In [ ]:
propose('write me code for openai')

In [ ]:


```python
import openai

openai.api_key = "YOUR_API_KEY"

response = openai.Completion.create(
  engine="text-davinci-003",
  prompt="Write me code for openai.",
  max_tokens=1024,
  temperature=0.7,
)

print(response.choices[0].text)
```

In [ ]:
import openai

openai.api_key = "YOUR_API_KEY"

def generate_text(prompt):
  response = openai.Completion.create(
    engine="text-davinci-003",
    prompt=prompt,
    max_tokens=1024,
    temperature=0.7,
  )
  return response.choices[0].text

# Example usage:
prompt = "Write a short story about a cat who goes on an adventure."
text = generate_text(prompt)
print(text)

In [ ]:
explain(7)

In [ ]:
propose("I need to find the 25 most signficantly upregulated genes from a dataframe called gene. Here are the column names: ENTREZID, SYMBOL, GENENAME, logFC AveExpr, t, P.Value, adj.P.Val")

In [ ]:


```python
import pandas as pd

# Assuming 'gene' is your DataFrame
top_25_upregulated = gene.sort_values(by='logFC', ascending=False).head(25)
```

In [ ]:
propose("help me develop a nextflow workflow")

In [ ]:


```groovy
// Define a workflow called 'my_workflow'
workflow my_workflow {

  // Define a channel called 'input_files' that will hold a list of input files
  Channel input_files = Channel.fromPath(['file1.txt', 'file2.txt', 'file3.txt'])

  // Define a process called 'preprocess' that takes an input file and performs some preprocessing
  process preprocess {

    // Input file
    input:
    file (input_file) from input_files

    // Output file
    output:
    file ('preprocessed_' + input_file.baseName + '.txt') into processed_files

    // Script to execute
    """
    # Preprocessing steps
    # ...

    # Write processed data to output file
    echo "Processed data" > preprocessed_${input_file.baseName}.txt
    """
  }

  // Define a process called 'analyze' that takes a preprocessed file and performs some analysis
  process analyze {

    // Input file
    input:
    file (processed_file) from processed_files

    // Output file
    output:
    file ('analyzed_' + processed_file.baseName + '.txt') into analyzed_files

    // Script to execute
    """
    # Analysis steps
    # ...

    # Write analysis results to output file
    echo "Analysis results" > analyzed_${processed_file.baseName}.txt
    """
  }

  // Define a process called 'report' that takes analyzed files and generates a report
  process report {

    // Input files
    input:
    file (analyzed_file) from analyzed_files

    // Output file
    output:
    file ('report.txt')

    // Script to execute
    """
    # Report generation steps
    # ...

    # Write report to output file
    echo "Report content" > report.txt
    """
  }
}
```

In [ ]:


```nextflow
// Define a parameter for the input file
params.inputFile = 'path/to/your/input.txt'

// Define a process to read the input file
process readInput {
  input:
    path(inputFile)

  output:
    file('input.txt')

  script:
    """
    cp ${inputFile} input.txt
    """
}

// Define a process to process the input file
process processFile {
  input:
    file(input)

  output:
    file('output.txt')

  script:
    """
    # Your processing logic here
    cat ${input} > output.txt
    """
}

// Define a process to write the output file
process writeOutput {
  input:
    file(output)

  output:
    file('final_output.txt')

  script:
    """
    cp ${output} final_output.txt
    """
}

// Connect the processes together
workflow {
  readInput(params.inputFile) | processFile | writeOutput
}
```